In [24]:
import logging
import re
import string
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

import mlflow
import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


## Data Overview

In [25]:
df = pd.read_csv('balanced_sentiment_dataset.csv')

In [26]:
df.head()

,sentiment,text
0,1,"And here is the rap song ""African Warrior Quee..."
1,1,"We asked chatGPT ,\n\nHow to become a successf..."
2,0,When I finally manage to convince myself to pe...
3,0,Although a healthier alternative to your typic...
4,1,"In my latest test, I asked #ChatGPT to write m..."


In [27]:
df.shape

(80000, 2)

In [28]:
df['sentiment'].value_counts()

sentiment
1    40000
0    40000
Name: count, dtype: int64

In [29]:
df.isnull().sum()

sentiment    0
text         0
dtype: int64

## Data Preprocessing

In [30]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_extra_whitespace(text):
    """Remove extra whitespace."""
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['text'] = df['text'].apply(lower_case)
        df['text'] = df['text'].apply(removing_urls)
        df['text'] = df['text'].apply(removing_numbers)
        df['text'] = df['text'].apply(removing_punctuations)
        df['text'] = df['text'].apply(remove_stop_words)
        df['text'] = df['text'].apply(lemmatization)
        df['text'] = df['text'].apply(remove_extra_whitespace)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [31]:
df = normalize_text(df)
df.head()

,sentiment,text
0,1,rap song african warrior queen chatgpt wrote l...
1,1,asked chatgpt n nhow become successful trader ...
2,0,finally manage convince peel sheet morning sec...
3,0,although healthier alternative typical vanilla...
4,1,latest test asked chatgpt write welcome letter...


In [32]:
all_words = ' '.join(df['text']).split()
# Convert to set to get unique words
unique_words = set(all_words)
print(f"Total unique words after preprocessing: {len(unique_words)}")

Total unique words after preprocessing: 98143


## Vectorization

In [33]:
vectorizer = CountVectorizer(max_features=20000)
X = vectorizer.fit_transform(df['text'])
y = df['sentiment']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow')
dagshub.init(repo_owner='rohitkr8527', repo_name='sentiment-analysis', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")


2025-09-02 10:11:18,557 - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as rohitkr8527

2025-09-02 10:11:18,565 - INFO - Accessing as rohitkr8527
2025-09-02 10:11:19,495 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/rohitkr8527/sentiment-analysis "HTTP/1.1 200 OK"
2025-09-02 10:11:20,304 - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "rohitkr8527/sentiment-analysis"

2025-09-02 10:11:20,308 - INFO - Initialized MLflow to track repo "rohitkr8527/sentiment-analysis"


Repository rohitkr8527/sentiment-analysis initialized!

2025-09-02 10:11:20,310 - INFO - Repository rohitkr8527/sentiment-analysis initialized!


<Experiment: artifact_location='mlflow-artifacts:/61da7ccc7aef45be883fbcabadf3f873', creation_time=1756453984341, experiment_id='0', last_update_time=1756453984341, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [36]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 20000)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  
        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-02 10:11:21,316 - INFO - Starting MLflow run...
2025-09-02 10:11:21,742 - INFO - Logging preprocessing parameters...
2025-09-02 10:11:22,984 - INFO - Initializing Logistic Regression model...
2025-09-02 10:11:22,985 - INFO - Fitting the model...
2025-09-02 10:11:25,314 - INFO - Model training complete.
2025-09-02 10:11:25,315 - INFO - Logging model parameters...
2025-09-02 10:11:25,737 - INFO - Making predictions...
2025-09-02 10:11:25,740 - INFO - Calculating evaluation metrics...
2025-09-02 10:11:25,751 - INFO - Logging evaluation metrics...
2025-09-02 10:11:27,396 - INFO - Saving and logging the model...
2025/09/02 10:11:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-09-02 10:11:34,946 - INFO - Model training and logging completed in 13.20 seconds.
2025-09-02 10:11:34,947 - INFO - Accuracy: 0.810125
2025-09-02 10:11:34,948 - INFO - Precisio

🏃 View run melodic-squirrel-667 at: https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow/#/experiments/0/runs/556abe59c8434556849ae7f1081ff8b5
🧪 View experiment at: https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow/#/experiments/0


In [37]:
# max_features = 5000